```
=================================================
Graded Challenge 2

Nama  : Muhammad Iqbal Saputra
Batch : RMT-032

Program ini dibuat untuk membuat 5 tabel 3NF untuk di proses dalam sql dan juga hasil dari sql tersebut di gunakan lagi untuk mencapai hasil yang ingin di dapatkan dan di munculkan lagi di ipynb
=================================================
```

#### 1. Data Exploration

###### a.

In [1]:
import pandas as pd 

# fungsi untuk mengimpor data dari file 'Financials.csv'
df = pd.read_csv("Financials.csv")

###### b.

In [2]:
# menampilkan data untuk mengecek apakah data telah berhasil diimpor
df.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$32,370.00",$-,"$32,370.00","$16,185.00","$16,185.00",1/1/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",$-,"$26,420.00","$13,210.00","$13,210.00",1/1/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",$-,"$32,670.00","$21,780.00","$10,890.00",1/6/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",$-,"$13,320.00","$8,880.00","$4,440.00",1/6/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",$-,"$37,050.00","$24,700.00","$12,350.00",1/6/2014,6,June,2014


*data masih belum bersih*

In [3]:
# menampilkan info dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Segment                700 non-null    object
 1   Country                700 non-null    object
 2    Product               700 non-null    object
 3    Discount Band         700 non-null    object
 4    Units Sold            700 non-null    object
 5    Manufacturing Price   700 non-null    object
 6    Sale Price            700 non-null    object
 7    Gross Sales           700 non-null    object
 8    Discounts             700 non-null    object
 9     Sales                700 non-null    object
 10   COGS                  700 non-null    object
 11   Profit                700 non-null    object
 12  Date                   700 non-null    object
 13  Month Number           700 non-null    int64 
 14   Month Name            700 non-null    object
 15  Year                   

*data memiliki 16 kolom, dan sebagian kolom yang seharusnya numerik belum menjadi tipe data numerik*

In [4]:
# menampilkan rangkuman data yang bersifat numerik
df.describe()

,Month Number,Year
count,700.000000,700.000000
mean,7.900000,2013.750000
std,3.377321,0.433322
min,1.000000,2013.000000
25%,5.750000,2013.750000
50%,9.000000,2014.000000
75%,10.250000,2014.000000
max,12.000000,2014.000000


*dikarnakan data belum bersih jadi hanya dapat data dari kolom month_number dan year*

###### c.

In [5]:
# Menampilkan list nama-nama kolom

list_nama_kolom = df.columns.tolist()
index = 1
for i in list_nama_kolom:
    print(f"{index}. {i}")
    index += 1

1. Segment
2. Country
3.  Product 
4.  Discount Band 
5.  Units Sold 
6.  Manufacturing Price 
7.  Sale Price 
8.  Gross Sales 
9.  Discounts 
10.   Sales 
11.  COGS 
12.  Profit 
13. Date
14. Month Number
15.  Month Name 
16. Year


*masih banyak trailing space*

###### d.

In [6]:
# Mencetak value unik dari setiap kolom
print("Segment:")
print(df['Segment'].unique())
print("\nCountry:")
print(df['Country'].unique())
print("\nProduct:")
print(df[' Product '].unique())
print("\nDiscount_Band:")
print(df[' Discount Band '].unique())

Segment:
['Government' 'Midmarket' 'Channel Partners' 'Enterprise' 'Small Business']

Country:
['Canada' 'Germany' 'France' 'Mexico' 'United States of America']

Product:
[' Carretera ' ' Montana ' ' Paseo ' ' Velo ' ' VTT ' ' Amarilla ']

Discount_Band:
[' None ' ' Low ' ' Medium ' ' High ']


- *berhasil mengeluarkan value unik dari kolom-kolom diatas namun trailing space belum dibersihkan*
- *segmentasi pasar dibagi menjadi 5 kategori, dan terdapat 5 negara, terdapat 6 produk, dan 4 tingkat diskon*


#### 2. Data Cleaning

###### a.

In [7]:
# Menghapus trailing space
df.columns = df.columns.str.strip()

# Mengganti nama kolom dengan nama yang sesuai
df = df.rename(columns={
    'Month Name': 'Month_Name',
    'Month Number': 'Month_Number',
    'Discount Band': 'Discount_Band',
    'Units Sold': 'Units_Sold',
    'Manufacturing Price': 'Manufacturing_Price',
    'Sale Price': 'Sale_Price',
    'Gross Sales': 'Gross_Sales'
})

# Hapus kolom yang tidak perlu
df = df.drop(columns=['Month_Number'])
df = df.drop(columns=['Month_Name'])
df = df.drop(columns=['Year'])


In [8]:
# Menampilkan daftar nama kolom
list_nama_kolom = df.columns.tolist()
index = 1
for i in list_nama_kolom:
    print(f"{index}. {i}")
    index += 1

1. Segment
2. Country
3. Product
4. Discount_Band
5. Units_Sold
6. Manufacturing_Price
7. Sale_Price
8. Gross_Sales
9. Discounts
10. Sales
11. COGS
12. Profit
13. Date


###### b.

In [9]:
# Fungsi untuk mengkonversi tanda '( )' pada value numerik menjadi minus
def convert_parentheses_to_negative(value):
    if '(' in value and ')' in value:
        return '-' + value.replace('(', '').replace(')', '')
    return value

# Mengaplikasikan fungsi
df['Profit'] = df['Profit'].astype(str).apply(convert_parentheses_to_negative)

# Menghapus karakter yg tidak perlu
df['Profit'] = pd.to_numeric(df['Profit'].str.replace('[$,]', '', regex=True), errors='coerce')

# mengkategorikan kolom-kolom numerik
numeric_columns = ['Units_Sold', 'Manufacturing_Price', 'Sale_Price', 'Gross_Sales','Discounts','Sales', 'COGS', 'Profit']

# menghapus trailing spaces pada kolom
df.columns = df.columns.str.strip()

# rubah kolom menjadi string terlebih dahulu
for col in numeric_columns:
    df[col] = df[col].astype(str)

# menghapus tanda $
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col].str.replace('[$,]', '', regex=True), errors='coerce')

*membersihkan data dari simbol dan spasi yg tidak diinginkan*


In [10]:
# cek apakah data sudah bersih
df.head()

,Segment,Country,Product,Discount_Band,Units_Sold,Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit,Date
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,NaN,32370.0,16185.0,16185.0,1/1/2014
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,NaN,26420.0,13210.0,13210.0,1/1/2014
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,NaN,32670.0,21780.0,10890.0,1/6/2014
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,NaN,13320.0,8880.0,4440.0,1/6/2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,NaN,37050.0,24700.0,12350.0,1/6/2014


In [11]:
# cek apakah tipe data sudah sesuai
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Segment              700 non-null    object 
 1   Country              700 non-null    object 
 2   Product              700 non-null    object 
 3   Discount_Band        700 non-null    object 
 4   Units_Sold           700 non-null    float64
 5   Manufacturing_Price  700 non-null    float64
 6   Sale_Price           700 non-null    float64
 7   Gross_Sales          700 non-null    float64
 8   Discounts            647 non-null    float64
 9   Sales                700 non-null    float64
 10  COGS                 700 non-null    float64
 11  Profit               695 non-null    float64
 12  Date                 700 non-null    object 
dtypes: float64(8), object(5)
memory usage: 71.2+ KB


###### c.

In [12]:
# handling missing value
df[numeric_columns] = df[numeric_columns].fillna(0)

In [13]:
# cek apakah missing value berhasil dihandle
df[numeric_columns].head()

,Units_Sold,Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit
0,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.0
1,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.0
2,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.0
3,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.0
4,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.0


In [14]:
# cek apakah missing value berhasil dihandle
df[numeric_columns].tail()

,Units_Sold,Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit
695,2475.0,260.0,300.0,742500.0,111375.00,631125.00,618750.0,12375.00
696,546.0,260.0,300.0,163800.0,24570.00,139230.00,136500.0,2730.00
697,1368.0,5.0,7.0,9576.0,1436.40,8139.60,6840.0,1299.60
698,723.0,10.0,7.0,5061.0,759.15,4301.85,3615.0,686.85
699,1806.0,250.0,12.0,21672.0,3250.80,18421.20,5418.0,13003.20


In [15]:
# cek data
df.head()

,Segment,Country,Product,Discount_Band,Units_Sold,Manufacturing_Price,Sale_Price,Gross_Sales,Discounts,Sales,COGS,Profit,Date
0,Government,Canada,Carretera,None,1618.5,3.0,20.0,32370.0,0.0,32370.0,16185.0,16185.0,1/1/2014
1,Government,Germany,Carretera,None,1321.0,3.0,20.0,26420.0,0.0,26420.0,13210.0,13210.0,1/1/2014
2,Midmarket,France,Carretera,None,2178.0,3.0,15.0,32670.0,0.0,32670.0,21780.0,10890.0,1/6/2014
3,Midmarket,Germany,Carretera,None,888.0,3.0,15.0,13320.0,0.0,13320.0,8880.0,4440.0,1/6/2014
4,Midmarket,Mexico,Carretera,None,2470.0,3.0,15.0,37050.0,0.0,37050.0,24700.0,12350.0,1/6/2014


In [16]:
# menambahkan index pada dataframe 
df['ID'] = range(1, len(df) + 1)
# menyimpan data yang sudah dibersihkan
df.to_csv('df.csv', index=False)

#### 3. Data Normalization

##### a. Normalisasi s/d 3NF

###### b. Membuat 5 tabel utk menyimpan hasil normalisasi 3NF

In [17]:
# membuat tabel segmentasi
segment_table = df[['Segment']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'segment_id', 'Segment': 'segment'})
segment_table['segment_id'] += 1


In [18]:
# membuat tabel negara
country_table = df[['Country']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'country_id', 'Country': 'country'})
country_table['country_id'] += 1


In [19]:
# membuat tabel produk
product_table = df[['Product', 'Manufacturing_Price']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'product_id', 'Product': 'product_name', 'Manufacturing_Price': 'manufacturing_price'})
product_table['product_id'] += 1


In [20]:
# membuat tabel diskon
discount_table = df[['Discount_Band']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index': 'discount_id', 'Discount_Band': 'discount_band'})
discount_table['discount_id'] += 1


###### C. Memberikan keterangan primary dan foreign key antar tabel

In [21]:
# mengatur agar kolom 'ID' berada disebelah kiri
df = df.reset_index().merge(segment_table, left_on='Segment', right_on='segment', how='left').merge(country_table, left_on='Country', right_on='country', how='left').merge(product_table, left_on='Product', right_on='product_name', how='left').merge(discount_table, left_on='Discount_Band', right_on='discount_band', how='left')

# tabel utama, ID -> primary key, segment_id -> foreign key, country_id -> foreign key, product_id -> foreign key, discount_id -> foreign key
sales_table = df[['ID', 'segment_id', 'country_id', 'product_id', 'discount_id', 'Units_Sold', 'Gross_Sales', 'Sales', 'COGS', 'Profit', 'Date']]


###### d. Menyimpan semua tabel


In [22]:
# menyimpan tabel-tabel yang sudah dinormalisasi
segment_table.to_csv('segment_table.csv', index=False)
country_table.to_csv('country_table.csv', index=False)
product_table.to_csv('product_table.csv', index=False)
discount_table.to_csv('discount_table.csv', index=False)
sales_table.to_csv('sales_table.csv', index=False)


#### 4. Relational Database & SQL

```SQL

-- 4.1
-- Membuat DATABASE

-- CREATE DATABASE gc2;

-- 4.2 
-- (TCL) 
BEGIN;

-- (DDL)
-- Membuat table segment
CREATE TABLE Segment (
    segment_id SERIAL PRIMARY KEY,
    segment VARCHAR(255) NOT NULL
);

-- Membuat table country
CREATE TABLE Country (
    country_id SERIAL PRIMARY KEY,
    country VARCHAR(255) NOT NULL
);

-- Membuat table product
CREATE TABLE Product (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    manufacturing_price FLOAT NOT NULL
);

-- Membuat table discount
CREATE TABLE Discount (
    discount_id SERIAL PRIMARY KEY,
    discount_band VARCHAR(255) NOT NULL
);

-- Membuat table sales
CREATE TABLE Sales (
    ID SERIAL PRIMARY KEY,
    segment_id INT REFERENCES Segment(segment_id),
    country_id INT REFERENCES Country(country_id),
    product_id INT REFERENCES Product(product_id),
    discount_id INT REFERENCES Discount(discount_id),
    units_sold FLOAT NOT NULL,
    gross_sales FLOAT NOT NULL,
    sales FLOAT NOT NULL,
    cogs FLOAT NOT NULL,
    profit FLOAT NOT NULL,
    date DATE NOT NULL
);

-- (DML)
-- Impor data segment dari segment_table.csv
COPY Segment(segment_id, segment)
FROM 'C:\tmp\segment_table.csv'
DELIMITER ','
CSV HEADER;

-- Impor data segment dari country_table.csv
COPY Country(country_id, country)
FROM 'C:\tmp\country_table.csv'
DELIMITER ','
CSV HEADER;

-- Impor data segment dari product_table.csv
COPY Product(product_id, product_name, manufacturing_price)
FROM 'C:\tmp\product_table.csv'
DELIMITER ','
CSV HEADER;

-- Impor data segment dari discount_table.csv
COPY Discount(discount_id, discount_band)
FROM 'C:\tmp\discount_table.csv'
DELIMITER ','
CSV HEADER;

-- Impor data segment dari sales_table.csv
COPY Sales(ID, segment_id, country_id, product_id, discount_id, units_sold, gross_sales, sales, cogs, profit, date)
FROM 'C:\tmp\sales_table.csv'
DELIMITER ','
CSV HEADER;

-- (DCL)
-- Pembuatan 2 user
CREATE USER admin_user WITH PASSWORD 'admin_password';
CREATE USER read_user WITH PASSWORD 'read_password';

-- Memberikan akses penuh ke database gc2
GRANT ALL PRIVILEGES ON DATABASE gc2 TO admin_user;

-- Memberikan akses untuk menggunakan schema public
GRANT USAGE ON SCHEMA public TO read_user;

-- Memberikan akses SELECT pada semua tabel di schema public
GRANT SELECT ON ALL TABLES IN SCHEMA public TO read_user;

-- Membatasi akses write (INSERT, UPDATE, DELETE, TRUNCATE) pada semua tabel di schema public
REVOKE INSERT, UPDATE, DELETE, TRUNCATE ON ALL TABLES IN SCHEMA public FROM read_user;

-- (TCL)
COMMIT;

#### 5. Pengujian Database


```sql
--5.1.a
CREATE TABLE segment_profit AS
SELECT Segment.segment, SUM(Sales.profit) AS total_profit
FROM Sales
JOIN Segment ON Sales.segment_id = Segment.segment_id
JOIN Discount ON Sales.discount_id = Discount.discount_id
WHERE Discount.discount_id = 1
GROUP BY Segment.segment
ORDER BY total_profit DESC;

SELECT *
FROM segment_profit
ORDER BY Total_profit DESC

--5.1.b
CREATE TABLE statistics_summary AS
SELECT Country.country, 
       AVG(Sales.sales) AS average_sales, 
       MIN(Sales.sales) AS min_sales, 
       MAX(Sales.sales) AS max_sales
FROM Sales
JOIN Country ON Sales.country_id = Country.country_id
GROUP BY Country.country;

SELECT * 
FROM statistics_summary
ORDER BY average_sales DESC
```

In [23]:
segment_profit = pd.read_csv('segment_profit.csv')
segment_profit

,segment,total_profit
0,Government,1117223
1,Small Business,294900
2,Channel Partners,166482
3,Midmarket,96850
4,Enterprise,61000


In [24]:
statistics_summary = pd.read_csv('statistics_summary.csv')
statistics_summary

,country,average_sales,min_sales,max_sales
0,United States of America,178784.501286,1730.54,1159200.0
1,Canada,177768.963500,1655.08,1038082.5
2,France,173958.373500,1763.86,962500.0
3,Germany,167895.291571,1685.60,1017338.0
4,Mexico,149638.229357,2508.66,848172.5


#### Kesimpulan


- Negara Amerika memiliki daya beli tertinggi dibanding 4 negara lainya
- Keuntungan terbesar dihasilkan oleh segmentasi pemerintahan dibanding segmentasi-segmentasi lainya
- Terbukti dari data, product paseo adalah produk terlaris diantar produk-produk lainya